In [ ]:
import os
import sys

import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager
from data_management.indicator_manager import IndicatorManager
from visualization.chart_manager import ChartManager
from data_management.preprocessor import DataPreprocessor

processor = DataPreprocessor()
dataset_manager = DatasetManager()
chart_manager = ChartManager()

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager
from data_management.indicator_manager import IndicatorManager
from visualization.chart_manager import ChartManager
from data_management.preprocessor import DataPreprocessor

processor = DataPreprocessor()
dataset_manager = DatasetManager()
chart_manager = ChartManager()

eur_1min_spirce = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/1min/EUR_USD.parquet'
df = pd.read_parquet(eur_1min_spirce)
indicator_manager = IndicatorManager()
resampled_df = df.resample('5min').agg({
                    'open': 'first',
                    'high': 'max',
                    'low': 'min',
                    'close': 'last',
                }).dropna()
df_with_indicators = indicator_manager.calculate_indicators(resampled_df, indicator_timeframe='1D')
# df_with_indicators.dropna(inplace=True)

# print(df_with_indicators)
df_norm = processor.normalize_simple(df=df_with_indicators)
df_with_indicators.to_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/EUR_USD_5T_indics_1D.parquet')
df_with_indicators

In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager

df_not_norm_dual_indic = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/EUR_USD_1H_indics_1D_and_1h.parquet'


df = pd.read_parquet(df_not_norm_dual_indic)
dataset_manager = DatasetManager()
train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)



saving_path = f'./logs/29nov/no_norm_1H_and_1D_dual_indic/'
os.makedirs(saving_path, exist_ok=True)

def make_train_env():
    env = ForexTradingEnv(
        df=train_df,
        pair='EUR_USD',

    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True)
    return env

def make_eval_env():
    env = ForexTradingEnv(

        df=val_df,
        pair='EUR_USD',
        # resample_interval='1h'
    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=False)
    env.training = False
    return env

train_env = make_train_env()
eval_env = make_eval_env()
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=saving_path,
    log_path=saving_path,
    eval_freq=50_000,  # Adjust as needed
    n_eval_episodes=5,
    deterministic=True,
    render=False
)

model = PPO(
    'MlpPolicy',
    train_env,
    verbose=0,
    tensorboard_log=f'{saving_path}tensorboard/',
)

model.learn(
    total_timesteps=3_000_000,  # Adjust as needed
    callback=eval_callback
)

model.save(f'{saving_path}best_model.zip')
train_env.save(f'{saving_path}vec_normalize.pkl')

EUR norm Hstack

In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager

pair = "EUR_USD"

eur_norm = Path('/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/normalized/EUR_USD.parquet')
df = pd.read_parquet(eur_norm)

dataset_manager = DatasetManager()
train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)



saving_path = f'./logs/27nov/norm_hstack/'
os.makedirs(saving_path, exist_ok=True)

def make_train_env():
    env = ForexTradingEnv(
        df=train_df,
        pair='EUR_USD',

    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True)
    return env

def make_eval_env():
    env = ForexTradingEnv(

        df=val_df,
        pair='EUR_USD',
        # resample_interval='1h'
    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=False)
    env.training = False
    return env

train_env = make_train_env()
eval_env = make_eval_env()
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=saving_path,
    log_path=saving_path,
    eval_freq=100_000,  # Adjust as needed
    n_eval_episodes=5,
    deterministic=True,
    render=False
)

model = PPO(
    'MlpPolicy',
    train_env,
    verbose=0,
    tensorboard_log=f'{saving_path}tensorboard/',
)

model.learn(
    total_timesteps=5_000_000,  # Adjust as needed
    callback=eval_callback
)

model.save(f'{saving_path}best_model.zip')
train_env.save(f'{saving_path}vec_normalize.pkl')

In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager

pair = "EUR_USD"

eur_norm = Path('/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/normalized/EUR_USD.parquet')
df = pd.read_parquet(eur_norm)
df.columns

In [ ]:

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from pathlib import Path
import pandas as pd
import os, sys
output_dir= "/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/normalized/"
Path(output_dir)
# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


from data_management.preprocessor import DataPreprocessor



processor = DataPreprocessor()

eur = pd.read_parquet("/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/EUR_USD.parquet")

eur_norm = processor.normalize_simple(df=eur)
eur_norm.to_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/normalized/EUR_USD.parquet')
eur_norm


In [ ]:
eur_norm.to_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/normalized/EUR_USD.parquet')